# R Data Cleaning Cheat Sheet

In [1]:
require(tidyverse)

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.5.2
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
data <- read_csv("scores.csv")

Rows: 4 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): participant_id, group
dbl (4): condition, test1, test2, test3

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## Handling missing data sentinels

Sometimes, special values (like -999) called *sentinels* are used to indicate missing data. The `replace` method can be used to replace these values with `None` to indicate that they are missing.

In [3]:
data

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
01,exp,1,6,9,-999
01,exp,2,4,8,9
02,con,1,9,10,8
02,con,2,7,9,7


To replace missing data sentinels in the `test3` column, using `na_if`.

In [4]:
data |>
  mutate(test3 = na_if(test3, -999))

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
01,exp,1,6,9,NA
01,exp,2,4,8,9
02,con,1,9,10,8
02,con,2,7,9,7


To replace missing data sentinels in multiple columns, use `across` to apply the `na_if` function to each column. The `~` indicates that the code after (here, `na_if(.x, -999))`) should be treated as a function, which will be applied to each of the indicated columns. The `.` is a placeholder for the column being evaluated.

In [5]:
data |>
  mutate(across(c(test1, test2, test3), ~ na_if(., -999)))

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
01,exp,1,6,9,NA
01,exp,2,4,8,9
02,con,1,9,10,8
02,con,2,7,9,7


Alternatively, we can set the `na` input when calling `read_csv` to set those special values to `null` immediately, in all columns. Here, setting `show_col_types` suppresses the messages that are usually shown when using `read_csv`.

In [6]:
data <- read_csv("scores.csv", na = "-999", show_col_types = FALSE)
data

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
01,exp,1,6,9,NA
01,exp,2,4,8,9
02,con,1,9,10,8
02,con,2,7,9,7


## Recoding variables

Use `recode` or `case_match` to change the values of variables and `as.numeric` or `as.character` to change the data type of variables.

In [7]:
data |>
  mutate(group = recode(group, "exp" = "Experimental", "con" = "Control"))

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
01,Experimental,1,6,9,NA
01,Experimental,2,4,8,9
02,Control,1,9,10,8
02,Control,2,7,9,7


In [8]:
data |>
  mutate(group = case_match(group, "exp" ~ "Experimental", "con" ~ "Control"))

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
01,Experimental,1,6,9,NA
01,Experimental,2,4,8,9
02,Control,1,9,10,8
02,Control,2,7,9,7


To change a variable into a different data type, use a type conversion function such as `as.numeric` or `as.character`.

In [9]:
data |>
  mutate(condition = as.character(condition))

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
01,exp,1,6,9,NA
01,exp,2,4,8,9
02,con,1,9,10,8
02,con,2,7,9,7


To change a numeric value to a string (known as a character vector in R), use `case_match`.

In [10]:
data |>
  mutate(condition = case_match(condition, 1 ~ "target", 2 ~ "lure"))

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
01,exp,target,6,9,NA
01,exp,lure,4,8,9
02,con,target,9,10,8
02,con,lure,7,9,7


## Working with missing data

Missing data should be marked as `NA` in DataFrames. They may then easily be excluded from calculations.

To count the number of `NA` values in each column, use `summarise_all` (which applies some summarizing function to all columns) with `~ sum(is.na(.))`, which sums the count of `NA` values for each column.

In [11]:
summarise_all(data, ~ sum(is.na(.)))

participant_id,group,condition,test1,test2,test3
<int>,<int>,<int>,<int>,<int>,<int>
0,0,0,0,0,1


Calculations will not ignore `NA` values by default.

In [12]:
data |> 
  summarise(across(c(test1, test2, test3), mean))

test1,test2,test3
<dbl>,<dbl>,<dbl>
6.5,9,NA


Calculations can be set to ignore `NA` values by setting `na.rm` to `TRUE`.

In [13]:
data |>
  summarise(across(c(test1, test2, test3), ~ mean(., na.rm = TRUE)))

test1,test2,test3
<dbl>,<dbl>,<dbl>
6.5,9,8


Alternatively, can replace `NA` values with some other value.

In [14]:
data |>
  mutate(across(c(test1, test2, test3), ~ replace_na(., 0)))

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
01,exp,1,6,9,0
01,exp,2,4,8,9
02,con,1,9,10,8
02,con,2,7,9,7


Use `drop_na` to completely exclude rows with `NA`.

In [15]:
data |>
  drop_na()

participant_id,group,condition,test1,test2,test3
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
01,exp,2,4,8,9
02,con,1,9,10,8
02,con,2,7,9,7
